***
### Парсинг геоданных

Парсить будем этот сайт - [geocode.maps.co](https://geocode.maps.co/). Возьмем очищенные данные (признаки ***street***, ***state*** и ***zipcode***):

In [1]:
import time
import requests

import pandas as pd

In [2]:
from warnings import filterwarnings
filterwarnings("ignore")

In [2]:
# данные для парсинга - это оригинальная таблица без дубликатов и пропусков c тремя столбцами - улица, штат и почтовый индекс
# города не очень хорошо подходят, почему-то этот сервис гораздо хуже работает

data_to_parse = pd.read_csv('data/data_to_parse.csv')
# data_to_parse['location'] = ''

In [3]:
for i in range(0, data_to_parse.shape[0]):
    street = data_to_parse['street'][i]
    # city = data_to_parse['city'][i]
    state = data_to_parse['state'][i]
    zipcode = data_to_parse['zipcode'][i]

    street = street.replace(' ', '+')
    # city = city.replace(' ', '+')

    api_url = f'https://geocode.maps.co/search?street={street}&state={state}&postalcode={zipcode}&country=US'
    response = requests.get(api_url)
    while response.status_code != 200:
        time.sleep(1)
        response = requests.get(api_url)
    
    data_to_parse.location.iloc[i] = response.json()

    print(f'Location {i} parsed...', end='\r')
    time.sleep(0.5)

    # пишем в файл каждые 1000 запросов
    if not i % 1000:
        data_to_parse.to_csv(f'data/data_parsed_{i}.csv')
        print(f'{i} locations parsed and saved.')

314000 locations parsed and saved.
315000 locations parsed and saved.
316000 locations parsed and saved.
317000 locations parsed and saved.
318000 locations parsed and saved.
319000 locations parsed and saved.
320000 locations parsed and saved.
321000 locations parsed and saved.
322000 locations parsed and saved.
323000 locations parsed and saved.
324000 locations parsed and saved.
325000 locations parsed and saved.
326000 locations parsed and saved.
327000 locations parsed and saved.
328000 locations parsed and saved.
329000 locations parsed and saved.
330000 locations parsed and saved.
331000 locations parsed and saved.
332000 locations parsed and saved.
333000 locations parsed and saved.
334000 locations parsed and saved.
335000 locations parsed and saved.
336000 locations parsed and saved.
337000 locations parsed and saved.
338000 locations parsed and saved.
339000 locations parsed and saved.
340000 locations parsed and saved.
341000 locations parsed and saved.
342000 locations par

In [ ]:
# data_to_parse.to_csv('data/data_parsed_final.csv')

In [3]:
# здесь по сути то же самое что и в 1-м парсинге, только тут адреса скорректированы - убраны номера квартир и прочая ненужная информация

# data_to_parse = pd.read_csv('data/data_to_parse_address_correct.csv')
# data_to_parse['location'] = ''

In [5]:
for i in range(42001, data_to_parse.shape[0]):
    street = data_to_parse['street_new'][i]
    # city = data_to_parse['city'][i]
    state = data_to_parse['state'][i]
    zipcode = data_to_parse['zipcode'][i]

    street = street.replace(' ', '+')
    # city = city.replace(' ', '+')
    if data_to_parse.street_new.iloc[i] != 'no_address':
        api_url = f'https://geocode.maps.co/search?street={street}&state={state}&postalcode={zipcode}&country=US'
        response = requests.get(api_url)
        j = 0.5
        while (response.status_code != 200) & (j <= 1.5):
            print(f'Code={response.status_code}. Wait for {j * 2:.2f} sec...........................', end='\r')
            time.sleep(j * 2)
            j += 0.25
            response = requests.get(api_url)
        if response.status_code == 200:
            data_to_parse.location.iloc[i] = response.json()
        else:
            data_to_parse.location.iloc[i] = 'timeout'
        print(f'Location {i} parsed. Code={response.status_code}.', end='\r')
        time.sleep(0.75)
    else:
        data_to_parse.location.iloc[i] = '[]'
    
    # пишем в файл каждые 1000 запросов
    if not i % 1000:
        data_to_parse.to_csv(f'data/data_parsed_{i}.csv')
        print(f'{i} locations parsed and saved.')

43000 locations parsed and saved......................
44000 locations parsed and saved......................
45000 locations parsed and saved......................
46000 locations parsed and saved......................
47000 locations parsed and saved......................
48000 locations parsed and saved......................
49000 locations parsed and saved......................
50000 locations parsed and saved......................
51000 locations parsed and saved......................


In [6]:
# data_to_parse.to_csv(f'data/data_parsed_{i}.csv')

К сожалению, этот сервис выдаёт несколько местоположений и зачастую самое значимое оказывается неверным, поэтому эту информацию мы не будем использовать. Данные сохранены в файле **data_local_info.csv**